# Отток клиентов банка

Из банка стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Описание данных
Признаки:

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак:

- Exited — факт ухода клиента

## <a id='content'>Содержание проекта</a>
1. <a href='#step-1'>Подготовка данных.</a>
2. <a href='#step-2'>Исследование задачи.</a>
3. <a href='#step-3'>Борьба с дисбалансом.</a>
4. <a href='#step-4'>Тестирование модели.</a>

##  <a id='step-1'>1. Подготовка данных</a>

In [6]:
# Подключим нужные модули.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle

# Создаём датафрейм из файла, в качестве индекса указываем столбец RowNumber,
# и выведем первые 10 строк вместе с информацией.
data = pd.read_csv('/datasets/Churn.csv', index_col=0)
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           9091 non-null   float64
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1.1+ MB


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0


Можно заметить, что в столбце *'Tenure'* присутствуют пропущенные данные. Скорее всего, информация не была заполнена для клиентов, не владеющих недвижимостью.

In [2]:
# Заполним нулями пропущенные значения.
data['Tenure'] = data['Tenure'].fillna(0)
data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<div class="alert alert-block alert-info">
<b>Совет: </b> Ты мог передать в метод '.fillna()' число, а не строку. А то у тебя происходит лишняя конвертация: float -> object -> int.
</div>
Исправил.

Проведём корелляционный анализ факторов, чтобы посмотреть, есть ли среди факторов сильная связь, и если есть, уберём часть из них.

Заодно заранее уберём столбцы *'CustomerId'* и *'Surname'*, эти признаки не будут использоваться из-за их "уникальности", они лишние для построения модели.

In [3]:
# Убираем "лишние" столбцы и выводим коэффициент корреляции для признаков.
data = data.drop(['CustomerId', 'Surname'], axis=1)
data.drop(['Exited'], axis=1).corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
CreditScore,1.000000,-0.003965,0.003087,0.006268,0.012238,-0.005458,0.025651,-0.001384
Age,-0.003965,1.000000,-0.007368,0.028308,-0.030680,-0.011721,0.085472,-0.007201
Tenure,0.003087,-0.007368,1.000000,-0.005821,0.010106,0.021387,-0.025856,0.011225
Balance,0.006268,0.028308,-0.005821,1.000000,-0.304180,-0.014858,-0.010084,0.012797
NumOfProducts,0.012238,-0.030680,0.010106,-0.304180,1.000000,0.003183,0.009612,0.014204
HasCrCard,-0.005458,-0.011721,0.021387,-0.014858,0.003183,1.000000,-0.011866,-0.009933
IsActiveMember,0.025651,0.085472,-0.025856,-0.010084,0.009612,-0.011866,1.000000,-0.011421
EstimatedSalary,-0.001384,-0.007201,0.011225,0.012797,0.014204,-0.009933,-0.011421,1.000000


Выполним прямле кодирование для признаков.

In [4]:
data_ohe = pd.get_dummies(data)
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
RowNumber,,,,,,,,,,,,,,
1,619,42,2.0,0.00,1,1,1,101348.88,1,1,0,0,1,0
2,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1,1,0
3,502,42,8.0,159660.80,3,1,0,113931.57,1,1,0,0,1,0
4,699,39,1.0,0.00,2,0,0,93826.63,0,1,0,0,1,0
5,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1,1,0


Укажем признаки и целевой признак, создадим обучающую, валидационную и тестовую выборки для модели.

In [5]:
features = data_ohe.drop(['Exited'], axis=1)
target = data_ohe['Exited']

# Создадим три выборки: для обучения, валидации и тестирования модели.
# Выделяем 20% данных для тестовой выборки 20% для валидационной и 60% для обучающей.
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.2, stratify=data[['Exited']], random_state=12345)

# Передаём часть данных от обучающей выборки для тестовой (0.25 * 0.8 = 0.2)
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, stratify=target_train, test_size = 0.25, random_state = 12345)

## <a id='step-2'>2. Исследование задачи</a>

Сравним частоту классов в целевом признаке и посмотрим, насколько сильно перекошен баланс.

In [6]:
data_ohe['Exited'].value_counts() / data_ohe.shape[0]

0    0.7963
1    0.2037
Name: Exited, dtype: float64

Класс "0" встречается гораздо чаще, чем класс "1".

Переберём гиперпараметры для обучения моделей без учёта веса классов, чтобы выяснить, с какими из них F1-мера будет выше всего.

In [7]:
# Обучим модель на обучающей выборке и подберём гиперпараметры на валидационной.

# Решающее дерево.
print('DecisionTreeClassifier F1-score:')
for depth in range(2,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    print(f'max_depth = {depth}: {f1_score(predicted, target_valid):.2f}')

# Случайный лес.
print('RandomForestClassifier F1-score:')
for estim in range(10, 51, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=10)
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    print(f'n_estimators = {estim}: {f1_score(predicted, target_valid):.2f}')

# Логистическая регрессия.    
print('LogisticRegression F1-score:')
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted = model.predict(features_valid)
print(f'{f1_score(predicted, target_valid):.2f}')

DecisionTreeClassifier F1-score:
max_depth = 2: 0.50
max_depth = 3: 0.40
max_depth = 4: 0.51
max_depth = 5: 0.48
max_depth = 6: 0.57
max_depth = 7: 0.57
max_depth = 8: 0.59
max_depth = 9: 0.54
max_depth = 10: 0.50
RandomForestClassifier F1-score:
n_estimators = 10: 0.55
n_estimators = 20: 0.55
n_estimators = 30: 0.53
n_estimators = 40: 0.53
n_estimators = 50: 0.54
LogisticRegression F1-score:
0.01


#### Вывод
Без учёта баланса классов, F1-мера получается ниже того, что нам нужно. Далее, попробуем учесть баланс классов при построении модели.

## <a id='step-3'>3. Борьба с дисбалансом</a>

В этот раз, учтём баланс классов. Изменим вес классов в соответствии с их соотношением (0.8 для класса 0 и 0.2 для класса 1). Одновременно с этим, проведём увеличение выборки и уменьшение выборки и посмотрим, как измениться F1-мера.

In [8]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

# Решающее дерево.
print('DecisionTreeClassifier F1-score:')
for depth in range(2,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight={0: 0.8, 1: 0.2})
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    print(f'max_depth = {depth}: {f1_score(target_valid, predicted_valid):.3f}')

# Случайный лес.
print('RandomForestClassifier F1-score:')
for estim in range(10, 101, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=10, class_weight={0: 0.8, 1: 0.2})
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    print(f'n_estimators = {estim}: {f1_score(target_valid, predicted_valid):.3f}')

# Логистическая регрессия.
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight={0: 0.8, 1: 0.2})
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print('LogisticRegression F1-score:')
print(f'{f1_score(target_valid, predicted_valid):.3f}')

DecisionTreeClassifier F1-score:
max_depth = 2: 0.521
max_depth = 3: 0.552
max_depth = 4: 0.565
max_depth = 5: 0.611
max_depth = 6: 0.605
max_depth = 7: 0.596
max_depth = 8: 0.552
max_depth = 9: 0.543
max_depth = 10: 0.539
RandomForestClassifier F1-score:
n_estimators = 10: 0.610
n_estimators = 20: 0.619
n_estimators = 30: 0.609
n_estimators = 40: 0.611
n_estimators = 50: 0.615
n_estimators = 60: 0.624
n_estimators = 70: 0.624
n_estimators = 80: 0.619
n_estimators = 90: 0.612
n_estimators = 100: 0.616
LogisticRegression F1-score:
0.412


In [9]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.02)

# Решающее дерево.
print('DecisionTreeClassifier F1-score:')
for depth in range(2,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight={0: 0.8, 1: 0.2})
    model.fit(features_downsampled, target_downsampled)
    predicted_valid = model.predict(features_valid)
    print(f'max_depth = {depth}: {f1_score(target_valid, predicted_valid):.2f}')

# Случайный лес.
print('RandomForestClassifier F1-score:')
for estim in range(10, 101, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=10, class_weight={0: 0.8, 1: 0.2})
    model.fit(features_downsampled, target_downsampled)
    predicted_valid = model.predict(features_valid)
    print(f'n_estimators = {estim}: {f1_score(target_valid, predicted_valid):.2f}')

# Логистическая регрессия.
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight={0: 0.8, 1: 0.2})
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print('LogisticRegression F1-score:')
print(f'{f1_score(target_valid, predicted_valid):.2f}')

DecisionTreeClassifier F1-score:
max_depth = 2: 0.43
max_depth = 3: 0.39
max_depth = 4: 0.42
max_depth = 5: 0.41
max_depth = 6: 0.41
max_depth = 7: 0.39
max_depth = 8: 0.39
max_depth = 9: 0.39
max_depth = 10: 0.39
RandomForestClassifier F1-score:
n_estimators = 10: 0.35
n_estimators = 20: 0.35
n_estimators = 30: 0.35
n_estimators = 40: 0.34
n_estimators = 50: 0.34
n_estimators = 60: 0.34
n_estimators = 70: 0.34
n_estimators = 80: 0.34
n_estimators = 90: 0.34
n_estimators = 100: 0.34
LogisticRegression F1-score:
0.36


#### Вывод
Как можно заметить, F1-мера увеличилась сильнее при увеличении выборки. Значит, отталкиваемся от этого. Также, модель на основе логистической регрессии показала F1-меру ниже, чем в случаях с моделями на основе решающего дерева и случайного леса.

Моделями с самой высокой F1-мерой оказались "решающее дерево" с max_depth равной 5 и "случайный лес" с n_estimators равным 60. Проверим их на тестовой выборке.

## <a id='step-4'>4. Тестирование модели</a>

Протестируем модели с помощью тестовой выборки.

In [10]:
model = DecisionTreeClassifier(random_state=12345, max_depth=5, class_weight={0: 5, 1: 1})
model.fit(features_downsampled, target_downsampled)
predicted_test = model.predict(features_test)
print(f'DecisionTreeClassifier F1-score: {f1_score(target_test, predicted_test):.3f}')

model = RandomForestClassifier(random_state=12345, n_estimators=60, max_depth=10, class_weight={0: 5, 1: 1})
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test)
print(f'RandomForestClassifier F1-score: {f1_score(target_test, predicted_test):.3f}')

DecisionTreeClassifier F1-score: 0.418
RandomForestClassifier F1-score: 0.617


Модель на основе решающего дерева не прошла порог в 0.59, в отличие от модели на основе случайного леса.

Дополнительно измерим AUC-ROC модели и сравним с F1-мерой.

In [11]:
model = RandomForestClassifier(random_state=12345, n_estimators=60, max_depth=10, class_weight={0: 5, 1: 1})
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test)
print(f'ROC-AUC score: {roc_auc_score(target_test, predicted_test):.3f}')

ROC-AUC score: 0.748


Значение больше того, что есть у случайной модели (0.5), но до идеального далеко.

<a href='#content'>Назад к содержанию</a>